In [ ]:
import pygame
import sys
import pandas as pd

In [ ]:
back = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\four_way_intersection.jpg')
data_path = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\predicted_traffic_times.xlsx'
car_image = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Down.png')  
# Load traffic signal images
red_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\red.png')  
green_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\green.png')  
yellow_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\yellow.png')

In [ ]:
# Function to read timer values from Excel file
def read_timers_from_excel(data_path):
    try:
        data = pd.read_excel(data_path)
        timer_1 = data["Predicted_Time_1"].tolist()  # Extract timer 1 values
        timer_2 = data["Predicted_Time_2"].tolist()  # Extract timer 2 values
        timer_3 = data["Predicted_Time_3"].tolist()  # Extract timer 3 values
        timer_4 = data["Predicted_Time_4"].tolist()  # Extract timer 4 values
        return timer_1, timer_2, timer_3, timer_4
    except FileNotFoundError:
        print("Excel file not found.")
    except Exception as e:
        print("An error occurred:", e)
    return [], [], [], []

In [ ]:
# Initialize pygame
pygame.init()

# Set the screen dimensions
screenW = 600
screenH = 600
screen = pygame.display.set_mode((screenW, screenH))

# Traffic signal coordinates
signalCoords = [(161, 106), (404, 106), (161, 404), (404, 404)]

# Set up initial position for the car
car_x = 300
car_y = 300

# Create a font for text rendering
font = pygame.font.Font(None, 24)

# Set up initial value for the timers
  # Path to your Excel file
timer_1, timer_2, timer_3, timer_4 = read_timers_from_excel(data_path)

# Main loop
running = True
row_index = 0  #Index to keep track of the current row in the Excel file
current_timer = 0  #Index of the current timer being active
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Clear the screen
    screen.fill((255, 255, 255))

    # Blit the car onto the screen
    screen.blit(car_image, (car_x, car_y))
    screen.blit(back,(0,0))
    
    # Read timer values from the current row
    timer_1, timer_2, timer_3, timer_4 = read_timers_from_excel(data_path)
    
    # Update and render timers
    timers = [timer_1, timer_2, timer_3, timer_4]
    for i, timer_list in enumerate(timers):
        if timer_list:  # Check if the timer list is not empty
            # print(i)
            timer = timer_list[i]  # Get the current timer value
            if i == current_timer:  # Check if this is the current active timer
                timer_text = font.render("Timer " + str(i + 1) + ": " + str(round(timer, 2)), True, (255, 0, 0))
            else:
                timer_text = font.render("Timer " + str(i + 1) + ": Paused", True, (255, 0, 0))
            if i % 2 == 0:  # Top corners
                screen.blit(timer_text, (20, 20 + i // 2 * (screenH - 50)))
            else:  # Bottom corners
                screen.blit(timer_text, (screenW - 200, 20 + (i // 2) * (screenH - 50)))

            # Decrement timer value if it is the current active timer
            if timer > 0 and i == current_timer:
                timer_list[i] -= 0.01

            # Check if the current timer has reached 0 and update the index of the current active timer
            if timer <= 0 and i == current_timer:
                current_timer = (current_timer + 1) % 4

            # Render traffic signals
            if i == current_timer:
                if timer > 3:  # More than 3 seconds left, display green signal
                    screen.blit(green_signal, signalCoords[i])
                elif timer > 1:  # More than 1 second left, display yellow signal
                    screen.blit(yellow_signal, signalCoords[i])
                else:  # Less than or equal to 1 second left, display red signal
                    screen.blit(red_signal, signalCoords[i])
            else:
                screen.blit(red_signal, signalCoords[i])  # Display red signal for inactive lanes


    # Update the display
    pygame.display.flip()

    # Add a small delay to control the frame rate
    pygame.time.delay(10)
    
    
pygame.quit()
sys.exit()
